<a href="https://colab.research.google.com/github/johntad110/Amharic-News-NLP/blob/main/AmharicPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!pip install pypdf2
!pip install transformers

In [55]:
import os
import PyPDF2
import pandas as pd
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from tensorflow.keras.optimizers import Adam

In [56]:
text_data = []

In [57]:
pdfs = '/content/'

In [58]:
for filename in os.listdir(pdfs):
  if filename.endswith('.pdf'):
    pdf_path = os.path.join(pdfs, filename)

    with open(pdf_path, 'rb') as file:
      reader = PyPDF2.PdfReader(file)
      print(f'extractin {pdf_path}')
      for page in reader.pages:
        text_data.append(page.extract_text())

extractin /content/01_genesis.pdf
extractin /content/the-bible-in-amharic.pdf


In [59]:
df = pd.DataFrame({'text': text_data})

In [68]:
# df2 = pd.read_csv('/content/Amharic News Dataset.csv')


# to ignore corrupt files...
file_path = '/content/Amharic News Dataset.csv'

import io

file_path = '/content/Amharic News Dataset.csv'

lines = []
with open(file_path, 'rb') as f:
    for line in f:
        try:
            decoded_line = line.decode('utf-8')
            lines.append(decoded_line)
        except UnicodeDecodeError:
            # Skip incomplete lines
            continue

df2 = pd.read_csv(io.StringIO('\n'.join(lines)))

In [70]:
df2.head()

,headline,category,date,views,article,link
0,የኦሊምፒክ ማጣሪያ ተሳታፊዎች የሚለዩበት ቻምፒዮና እየተካሄደ ይገኛል,ስፖርት,"January 14, 2021",2,ብርሃን ፈይሳየኢትዮጵያ ቦክስ ፌዴሬሽን በየዓመቱ የሚያዘጋጀው የክለቦች ቻ...,https://www.press.et/Ama/?p=39481
1,አዲስ ዘመን ድሮ,መዝናኛ,"December 28, 2020",4,የአዲስ ዘመን ጋዜጣ ቀደምት ዘገባዎች በእጅጉ ተነባቢ ዛሬም ላገኛቸው በ...,https://www.press.et/Ama/?p=38334
2,የአረንጓዴ ጎርፍ በጎ አድራጎት አምባሳደሮች ተሰየሙ,ስፖርት,"January 6, 2021",6,ቦጋለ አበበየአዲስ አበባ ከተማ አስተዳደር ስፖርት ኮሚሽን ከኢትዮጵያ አረ...,https://www.press.et/Ama/?p=39010
3,የሊጉ በቢዝነስ ሞዴል መመራት አበረታች ጅምር መሆኑ ተገለጸ,ስፖርት,"January 6, 2021",5,ብርሃን ፈይሳአዲስ አበባ፡- የኢትዮጵያ ፕሪምየር ሊግ በሼር ካምፓኒ እንዲ...,https://www.press.et/Ama/?p=39011
4,የኦሊምፒክ ሥራ አስፈፃሚው እስከ ቶኪዮ ኦሊምፒክ ማግስት ይቀጥላል,ስፖርት,"January 6, 2021",12,ቦጋለ አበበ የኢትዮጵያ ኦሊምፒክ ኮሚቴ አርባ አምስተኛ መደበኛ ጠቅላላ ጉ...,https://www.press.et/Ama/?p=39012


In [71]:
# load pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'
model = TFGPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [72]:
# tokenize and decode the df
tokenized_dataset = df['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

Token indices sequence length is longer than the specified maximum sequence length for this model (3923 > 1024). Running this sequence through the model will result in indexing errors


In [73]:
device = tf.device("cuda" if tf.config.list_physical_devices('GPU') else "cpu")

In [ ]:
input_ids = [ids for ids in tokenized_dataset]
train_dataset = tf.data.Dataset.from_tensor_slices(input_ids)
train_dataset = train_dataset.shuffle(len(input_ids)).batch(4)

In [ ]:
# freeze pretrained layers
for layer in model.layers:
  layer.trainable = False

In [ ]:
# add task specific layers
dense_layer = tf.keras.layers.Dense(model.config.vocab_size)
model.layers[-1] = dense_layer

In [ ]:
# training setup
optimizer = Adam(learning_rate=1e-4)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# training
for epoch in range(1):
  total_loss = 0
  for batch in train_dataset:
    with tf.GradientTape() as tape:
      outputs = model(batch, training=True)
      logits = outputs.logits
      loss = loss_function(batch[:, 1:], logits[:, :-1, :])

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    total_loss += loss

  print(f"Epoch {epoch + 1} Loss: {total_loss / len(train_dataset)}")

In [ ]:
model.save_pretrained('amharic_gpt2_finetuned')
tokenizer.save_pretrained('amharic_gpt2_finetuned')